## 0.Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

###################################
from sklearn import preprocessing
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import shuffle

#####################################
sns.set_style('whitegrid')

font = {'weight' : 'bold',
        'size'   : 14}

%matplotlib inline
#####################################
# SET PATH
PATH = 'Dataset/NSL-KDD/'

## 1.Datasets

In [2]:
df = pd.read_csv(PATH+'combined_df.csv')
df_test = pd.read_csv(PATH+'combined_test_df.csv')
nor_df = pd.read_csv(PATH+'normal_df.csv')

**Train dataframe**

In [3]:
df.head()

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_REJ,flag_RSTO,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH
0,0,491,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0,146,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,0,232,8153,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
4,0,199,420,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


**TEST Dataframe**

In [4]:
df_test.head()

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_REJ,flag_RSTO,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH
0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,2,12983,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,20,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,1,0,15,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [5]:
nor_df.head()

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,REJ,RSTO,RSTOS0,RSTR,S0,S1,S2,S3,SF,SH
0,0,491,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0,146,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0,232,8153,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,0,199,420,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
4,0,287,2251,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


**Features names**

In [6]:
[print() if i%10 == 0 else print(j, end = ', ') for i,j in enumerate(df.columns)]
print('\n','*-'*50)
[print() if i%10 == 0 else print(j, end = ', ') for i,j in enumerate(df_test.columns)]
''


src_bytes, dst_bytes, land, wrong_fragment, urgent, hot, num_failed_logins, logged_in, num_compromised, 
su_attempted, num_root, num_file_creations, num_shells, num_access_files, num_outbound_cmds, is_host_login, is_guest_login, count, 
serror_rate, srv_serror_rate, rerror_rate, srv_rerror_rate, same_srv_rate, diff_srv_rate, srv_diff_host_rate, dst_host_count, dst_host_srv_count, 
dst_host_diff_srv_rate, dst_host_same_src_port_rate, dst_host_srv_diff_host_rate, dst_host_serror_rate, dst_host_srv_serror_rate, dst_host_rerror_rate, dst_host_srv_rerror_rate, label, protocol_icmp, 
protocol_udp, service_IRC, service_X11, service_Z39_50, service_aol, service_auth, service_bgp, service_courier, service_csnet_ns, 
service_daytime, service_discard, service_domain, service_domain_u, service_echo, service_eco_i, service_ecr_i, service_efs, service_exec, 
service_ftp, service_ftp_data, service_gopher, service_harvest, service_hostnames, service_http, service_http_2784, service_http_443, service_

''

### 1.x Different dataset for different categories. 
<br>
This will aid to create classification models which are independent of each others. 

In [7]:
# 0=normal, 1=DoS, 2=Probe, 3=R2L and 4=U2R.
normal_df = df[df['label'] == 0]

dos_df = df[df['label'] == 1]
dos_df = pd.concat([dos_df,normal_df])
dos_df = shuffle(dos_df)
probe_df = df[df['label'] == 2]
probe_df = pd.concat([probe_df,normal_df])
probe_df = shuffle(probe_df)
r2l_df = df[df['label'] == 3]
r2l_df = pd.concat([r2l_df,normal_df])
r2l_df = shuffle(r2l_df)
u2r_df = df[df['label'] == 4]
u2r_df = pd.concat([u2r_df,normal_df])
u2r_df = shuffle(u2r_df)
# TEST DATASET

Normal_df_test = df_test[df_test['label'] == 0]

Dos_df_test = df_test[df_test['label'] == 1]
Dos_df_test = pd.concat([Normal_df_test,Dos_df_test])
Dos_df_test = shuffle(Dos_df_test)
Probe_df_test = df_test[df_test['label'] == 2]
Probe_df_test = pd.concat([Normal_df_test,Probe_df_test])
Probe_df_test = shuffle(Probe_df_test)
R2L_df_test = df_test[df_test['label'] == 3]
R2L_df_test = pd.concat([Normal_df_test,R2L_df_test])
R2L_df_test = shuffle(R2L_df_test)
U2R_df_test = df_test[df_test['label'] == 4]
U2R_df_test = pd.concat([Normal_df_test,U2R_df_test])
U2R_df_test = shuffle(U2R_df_test)


## 1.3 Save the dataset (updated)

In [8]:
'''
# Train dataset!

dos_df.to_csv(PATH+'dos_df_train.csv',index = False)
probe_df.to_csv(PATH+'probe_df_train.csv',index = False)
r2l_df.to_csv(PATH+'r2l_df_train.csv',index = False)
u2r_df.to_csv(PATH+'u2r_df_train.csv',index = False)

# Test Dataset!

Dos_df_test.to_csv(PATH+'dos_df_test.csv',index = False)
Probe_df_test.to_csv(PATH+'probe_df_test.csv',index = False)
R2L_df_test.to_csv(PATH+'r2l_df_test.csv',index = False)
U2R_df_test.to_csv(PATH+'u2r_df_test.csv',index = False)
'''
;


''

## 2. Preprocessing

In [9]:
print('Dimensions of Normal:', normal_df.shape)
print('Train:')
print('Dimensions of DoS:' ,dos_df.shape)
print('Dimensions of Probe:' ,probe_df.shape)
print('Dimensions of R2L:' ,r2l_df.shape)
print('Dimensions of U2R:' ,u2r_df.shape)
print('--.'*20)
print('Dimensions of test Normal:', Normal_df_test.shape)
print('Test:')
print('Reset')
print('Dimensions of DoS:' ,Dos_df_test.shape)
print('Dimensions of Probe:' ,Probe_df_test.shape)
print('Dimensions of R2L:' ,R2L_df_test.shape)
print('Dimensions of U2R:' ,U2R_df_test.shape)

Dimensions of Normal: (67343, 123)
Train:
Dimensions of DoS: (113270, 123)
Dimensions of Probe: (78999, 123)
Dimensions of R2L: (68338, 123)
Dimensions of U2R: (67395, 123)
--.--.--.--.--.--.--.--.--.--.--.--.--.--.--.--.--.--.--.--.
Dimensions of test Normal: (9711, 123)
Test:
Reset
Dimensions of DoS: (17171, 123)
Dimensions of Probe: (12132, 123)
Dimensions of R2L: (12596, 123)
Dimensions of U2R: (9778, 123)


In [10]:
X_DoS = dos_df.drop('label',1)
Y_DoS = dos_df.label
X_Probe = probe_df.drop('label',1)
Y_Probe = probe_df.label
X_R2L = r2l_df.drop('label',1)
Y_R2L = r2l_df.label
X_U2R = u2r_df.drop('label',1)
Y_U2R = u2r_df.label

# test set
X_DoS_test = Dos_df_test.drop('label',1)
Y_DoS_test = Dos_df_test.label
X_Probe_test = Probe_df_test.drop('label',1)
Y_Probe_test = Probe_df_test.label
X_R2L_test = R2L_df_test.drop('label',1)
Y_R2L_test = R2L_df_test.label
X_U2R_test = U2R_df_test.drop('label',1)
Y_U2R_test = U2R_df_test.label


In [11]:
col_names  = X_DoS.columns

In [12]:
Normal_df_test.shape

(9711, 123)

In [13]:
X_DoS.shape

(113270, 122)

### 2.1 Scaler

In [14]:
scaler1 = preprocessing.StandardScaler().fit(X_DoS)
X_DoS=scaler1.transform(X_DoS) 
scaler2 = preprocessing.StandardScaler().fit(X_Probe)
X_Probe=scaler2.transform(X_Probe) 
scaler3 = preprocessing.StandardScaler().fit(X_R2L)
X_R2L=scaler3.transform(X_R2L) 
scaler4 = preprocessing.StandardScaler().fit(X_U2R)
X_U2R=scaler4.transform(X_U2R) 

/opt/Anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/opt/Anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  
/opt/Anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/opt/Anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  after removing the cwd from sys.path.
/opt/Anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float

In [15]:
print(X_DoS.std(axis=0))

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 0. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1.
 1. 1.]


# 3.Univariate feature selection

In [16]:
probe_df.columns[56]

'service_eco_i'

In [17]:
df.columns[56]

'service_eco_i'

In [18]:
'label' in col_names

False

In [19]:
np.seterr(divide='ignore', invalid='ignore');
selector = SelectPercentile(f_classif, percentile=10)

In [20]:
def get_univariate_features(X, y, col = col_names):
    x_ids = selector.fit_transform(X,y)
    true = selector.get_support()
    newcolindex_=[i for i, x in enumerate(true) if x]
    newcolname_=list(col[i] for i in newcolindex_)
    return newcolname_,x_ids

>**DOS**

In [21]:
Dos_uf_col_train, Dos_uf_train = get_univariate_features(X_DoS, Y_DoS)
Dos_uf_col_train

/opt/Anaconda3/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [ 16  44  63  66  68  86 114] are constant.
  UserWarning)


['logged_in',
 'count',
 'serror_rate',
 'srv_serror_rate',
 'same_srv_rate',
 'dst_host_count',
 'dst_host_srv_count',
 'dst_host_same_srv_rate',
 'dst_host_serror_rate',
 'dst_host_srv_serror_rate',
 'service_http',
 'flag_S0',
 'flag_SF']

> **Probe**

In [22]:
Probe_uf_col_train, Probe_uf_train = get_univariate_features(X_Probe, Y_Probe)
Probe_uf_col_train

/opt/Anaconda3/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [ 4 16] are constant.
  UserWarning)


['logged_in',
 'rerror_rate',
 'srv_rerror_rate',
 'dst_host_srv_count',
 'dst_host_diff_srv_rate',
 'dst_host_same_src_port_rate',
 'dst_host_srv_diff_host_rate',
 'dst_host_rerror_rate',
 'dst_host_srv_rerror_rate',
 'protocol_icmp',
 'service_eco_i',
 'service_private',
 'flag_SF']

> **R2L**

In [23]:
R2L_uf_col_train, R2L_uf_train = get_univariate_features(X_R2L, Y_R2L)
R2L_uf_col_train

/opt/Anaconda3/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [  4  16  43  44  46  47  48  49  50  51  54  57  58  62  63  64  66  67
  68  70  71  72  73  74  76  77  78  79  80  81  82  83  86  87  89  92
  93  96  98  99 100 107 108 109 110 114] are constant.
  UserWarning)


['src_bytes',
 'dst_bytes',
 'hot',
 'num_failed_logins',
 'is_guest_login',
 'dst_host_srv_count',
 'dst_host_same_src_port_rate',
 'dst_host_srv_diff_host_rate',
 'service_ftp',
 'service_ftp_data',
 'service_http',
 'service_imap4',
 'flag_RSTO']

> **U2R**

In [24]:
U2R_uf_col_train, U2R_uf_train = get_univariate_features(X_U2R, Y_U2R)
U2R_uf_col_train

/opt/Anaconda3/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [  4  16  43  44  46  47  48  49  50  51  54  57  58  62  63  64  66  67
  68  70  71  72  73  74  75  76  77  78  79  80  81  82  83  86  87  89
  92  93  96  98  99 100 107 108 109 110 114] are constant.
  UserWarning)


['urgent',
 'hot',
 'root_shell',
 'num_file_creations',
 'num_shells',
 'srv_diff_host_rate',
 'dst_host_count',
 'dst_host_srv_count',
 'dst_host_same_src_port_rate',
 'dst_host_srv_diff_host_rate',
 'service_ftp_data',
 'service_http',
 'service_telnet']

## Results from UFS

In [25]:
print('Features :: DoS ')
print('-----'*10)
print('\n'.join(Dos_uf_col_train),'\n')
print('=*=*='*10,'\n')
      
print('Features :: Probe ')
print('-----'*10)
print('\n'.join(Probe_uf_col_train),'\n')
print('=*=*='*10,'\n')
      
print('Features :: R2L')
print('-----'*10)
print('\n'.join(R2L_uf_col_train),'\n')
print('=*=*='*10,'\n')
      
print('Features :: U2R')
print('-----'*10)
print('\n'.join(U2R_uf_col_train),'\n')
print('=*=*='*10,'\n')

Features :: DoS 
--------------------------------------------------
logged_in
count
serror_rate
srv_serror_rate
same_srv_rate
dst_host_count
dst_host_srv_count
dst_host_same_srv_rate
dst_host_serror_rate
dst_host_srv_serror_rate
service_http
flag_S0
flag_SF 

=*=*==*=*==*=*==*=*==*=*==*=*==*=*==*=*==*=*==*=*= 

Features :: Probe 
--------------------------------------------------
logged_in
rerror_rate
srv_rerror_rate
dst_host_srv_count
dst_host_diff_srv_rate
dst_host_same_src_port_rate
dst_host_srv_diff_host_rate
dst_host_rerror_rate
dst_host_srv_rerror_rate
protocol_icmp
service_eco_i
service_private
flag_SF 

=*=*==*=*==*=*==*=*==*=*==*=*==*=*==*=*==*=*==*=*= 

Features :: R2L
--------------------------------------------------
src_bytes
dst_bytes
hot
num_failed_logins
is_guest_login
dst_host_srv_count
dst_host_same_src_port_rate
dst_host_srv_diff_host_rate
service_ftp
service_ftp_data
service_http
service_imap4
flag_RSTO 

=*=*==*=*==*=*==*=*==*=*==*=*==*=*==*=*==*=*==*=*= 

Features

## 4. Recursive Feature elimination

 Internet helped here!!
 https://machinelearningmastery.com/feature-selection-machine-learning-python/

In [26]:
clf = DecisionTreeClassifier(random_state=0)

#rank all features
rfe = RFE(clf, n_features_to_select=1)


## 4.1 Ranking of features Obtained from UFS

In [27]:
rfe.fit(Dos_uf_train, Y_DoS)
print ("DoS Features sorted by their rank:")
print (sorted(zip(map(lambda x: round(x, 4), rfe.ranking_), Dos_uf_col_train)))

DoS Features sorted by their rank:
[(1, 'same_srv_rate'), (2, 'count'), (3, 'flag_SF'), (4, 'dst_host_serror_rate'), (5, 'dst_host_same_srv_rate'), (6, 'dst_host_srv_count'), (7, 'dst_host_count'), (8, 'logged_in'), (9, 'serror_rate'), (10, 'dst_host_srv_serror_rate'), (11, 'srv_serror_rate'), (12, 'service_http'), (13, 'flag_S0')]


In [28]:
rfe.fit(Probe_uf_train, Y_Probe)
print ("Probe Features sorted by their rank:")
print (sorted(zip(map(lambda x: round(x, 4), rfe.ranking_), Probe_uf_col_train)))

Probe Features sorted by their rank:
[(1, 'dst_host_same_src_port_rate'), (2, 'dst_host_srv_count'), (3, 'dst_host_rerror_rate'), (4, 'service_private'), (5, 'logged_in'), (6, 'dst_host_diff_srv_rate'), (7, 'dst_host_srv_diff_host_rate'), (8, 'flag_SF'), (9, 'service_eco_i'), (10, 'rerror_rate'), (11, 'protocol_icmp'), (12, 'dst_host_srv_rerror_rate'), (13, 'srv_rerror_rate')]


In [29]:
rfe.fit(R2L_uf_train, Y_R2L)
print ("R2L Features sorted by their rank:")
print (sorted(zip(map(lambda x: round(x, 4), rfe.ranking_), R2L_uf_col_train)))

R2L Features sorted by their rank:
[(1, 'src_bytes'), (2, 'dst_bytes'), (3, 'hot'), (4, 'dst_host_srv_diff_host_rate'), (5, 'service_ftp_data'), (6, 'dst_host_same_src_port_rate'), (7, 'dst_host_srv_count'), (8, 'num_failed_logins'), (9, 'service_imap4'), (10, 'is_guest_login'), (11, 'service_ftp'), (12, 'flag_RSTO'), (13, 'service_http')]


In [30]:
rfe.fit(U2R_uf_train, Y_U2R)
print ("U2R Features sorted by their rank:")
print (sorted(zip(map(lambda x: round(x, 4), rfe.ranking_), U2R_uf_col_train)))

U2R Features sorted by their rank:
[(1, 'hot'), (2, 'dst_host_srv_count'), (3, 'dst_host_count'), (4, 'root_shell'), (5, 'num_shells'), (6, 'service_ftp_data'), (7, 'dst_host_srv_diff_host_rate'), (8, 'num_file_creations'), (9, 'dst_host_same_src_port_rate'), (10, 'service_telnet'), (11, 'srv_diff_host_rate'), (12, 'service_http'), (13, 'urgent')]


## 4.2 From complete dataset

In [31]:
X_DoS.std(axis = 0)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1.,
       0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1.,
       1., 1., 1.])

In [32]:
# Use some other classifier.
clf = DecisionTreeClassifier(random_state=0)
rfe = RFE(estimator=clf, n_features_to_select=13, step=1)

def get_rfe(rfe,X,y, col = col_names):
    rfe.fit(X, y)
    rfe_idx = rfe.transform(X)
    true = rfe.support_
    rfecolindex_ = [i for i, x in enumerate(true) if x]
    rfecolname_ = list(col[i] for i in rfecolindex_)
    return rfe_idx, rfecolname_


    

### RFE for DOS taking all the 122 features.

In [33]:
%%time 
RFE_dos, RFE_dos_col = get_rfe(rfe, X_DoS, Y_DoS)


CPU times: user 1min 15s, sys: 515 ms, total: 1min 16s
Wall time: 1min 16s


In [34]:
RFE_dos_col

['src_bytes',
 'dst_bytes',
 'wrong_fragment',
 'num_compromised',
 'same_srv_rate',
 'diff_srv_rate',
 'dst_host_count',
 'dst_host_same_srv_rate',
 'dst_host_serror_rate',
 'dst_host_srv_serror_rate',
 'service_ecr_i',
 'flag_RSTR',
 'flag_S0']

**In the sameway anyone run for all the other dataset.**
<br>
Just call the function **get_rfe** for the other types of attack and summarize the results.
use the updated dataset. Columns returned from UFS, RFE or some other method which we **MAY** develop if we develop could be used with the datasets saved. 

In [35]:
RFE_probe, RFE_probe_col = get_rfe(rfe, X_Probe, Y_Probe)

In [36]:
RFE_probe_col

['src_bytes',
 'dst_bytes',
 'rerror_rate',
 'dst_host_same_srv_rate',
 'dst_host_diff_srv_rate',
 'dst_host_same_src_port_rate',
 'dst_host_rerror_rate',
 'service_finger',
 'service_ftp_data',
 'service_http',
 'service_private',
 'service_smtp',
 'service_telnet']

In [37]:
RFE_R2L, RFE_R2L_col = get_rfe(rfe, X_R2L, Y_R2L)

In [38]:
RFE_R2L_col

['duration',
 'src_bytes',
 'dst_bytes',
 'hot',
 'num_failed_logins',
 'num_access_files',
 'dst_host_count',
 'dst_host_srv_count',
 'dst_host_same_srv_rate',
 'dst_host_same_src_port_rate',
 'dst_host_srv_diff_host_rate',
 'service_ftp_data',
 'service_imap4']

In [39]:
RFE_U2R, RFE_U2R_col = get_rfe(rfe, X_U2R, Y_U2R)

In [40]:
RFE_U2R_col

['duration',
 'src_bytes',
 'dst_bytes',
 'hot',
 'root_shell',
 'num_file_creations',
 'num_shells',
 'srv_count',
 'dst_host_count',
 'dst_host_same_srv_rate',
 'dst_host_srv_diff_host_rate',
 'service_ftp_data',
 'service_other']

In [41]:
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
import xgboost

In [42]:
from time import time

In [43]:
import warnings
warnings.filterwarnings("ignore")

In [44]:
rf = RandomForestClassifier(random_state=0)
MLPc = MLPClassifier(random_state=0)
mods = [clf, rf]

Could only use for two algorithms... tested others.. either they took too long or they couldn't resolve important features

In [45]:
RFE_dos, RFE_probe, RFE_R2L, RFE_U2R, RFE_dos_col, RFE_probe_col, RFE_R2L_col, RFE_U2R_col = [[[]] for i in range(8)]

for i, cla in enumerate(mods):
    print("Iteration " + str(i + 1) + " of " + str(len(mods))+ " :", end =" ")
    t0 = time()
    rfe = RFE(estimator=cla, n_features_to_select=13, step=1)
    print("1%..", end ="")
    x, y = get_rfe(rfe, X_DoS, Y_DoS)
    RFE_dos.append(x)
    RFE_dos_col.append(y)
    print("..25%..", end ="")
    x, y = get_rfe(rfe, X_Probe, Y_Probe)
    RFE_probe.append(x)
    RFE_probe_col.append(y)
    print("..50%..", end="")
    x, y = get_rfe(rfe, X_R2L, Y_R2L)
    RFE_R2L.append(x)
    RFE_R2L_col.append(y)
    print("..75%..", end ="")
    x, y = get_rfe(rfe, X_U2R, Y_U2R)
    RFE_U2R.append(x)
    RFE_U2R_col.append(y)
    print("..100%",)
    print("Time taken {} seconds\n".format(time() - t0))

Iteration 1 of 2 : 1%....25%....50%....75%....100%
Time taken 184.99820375442505 seconds

Iteration 2 of 2 : 1%....25%....50%....75%....100%
Time taken 178.26801991462708 seconds



In [46]:
dos_fet = list(set(RFE_dos_col[1]).union(set(RFE_dos_col[2]), set(Dos_uf_col_train)))
dos_fet

['num_compromised',
 'srv_serror_rate',
 'src_bytes',
 'diff_srv_rate',
 'flag_SF',
 'service_private',
 'service_ecr_i',
 'wrong_fragment',
 'count',
 'serror_rate',
 'dst_host_count',
 'dst_bytes',
 'same_srv_rate',
 'dst_host_srv_count',
 'flag_RSTR',
 'flag_S0',
 'dst_host_serror_rate',
 'service_http',
 'dst_host_same_srv_rate',
 'logged_in',
 'protocol_icmp',
 'dst_host_srv_serror_rate']

In [47]:
probe_fet = list(set(RFE_probe_col[2]).union(set(RFE_probe_col[1]), set(Probe_uf_col_train)))
probe_fet

['src_bytes',
 'srv_rerror_rate',
 'dst_host_srv_rerror_rate',
 'flag_SF',
 'service_telnet',
 'service_private',
 'dst_host_srv_diff_host_rate',
 'dst_host_rerror_rate',
 'service_finger',
 'service_ftp_data',
 'service_smtp',
 'dst_host_count',
 'service_eco_i',
 'dst_bytes',
 'protocol_tcp',
 'dst_host_srv_count',
 'service_http',
 'dst_host_same_srv_rate',
 'logged_in',
 'rerror_rate',
 'protocol_icmp',
 'dst_host_diff_srv_rate',
 'dst_host_same_src_port_rate']

In [48]:
R2L_fet = list(set(RFE_dos_col[2]).union(set(RFE_dos_col[1]), set(R2L_uf_col_train)))
R2L_fet

['num_compromised',
 'src_bytes',
 'diff_srv_rate',
 'flag_RSTO',
 'flag_SF',
 'service_private',
 'dst_host_srv_diff_host_rate',
 'dst_host_same_src_port_rate',
 'service_ecr_i',
 'hot',
 'service_imap4',
 'wrong_fragment',
 'service_ftp_data',
 'count',
 'serror_rate',
 'dst_host_count',
 'num_failed_logins',
 'service_ftp',
 'dst_bytes',
 'is_guest_login',
 'same_srv_rate',
 'dst_host_srv_count',
 'flag_RSTR',
 'flag_S0',
 'dst_host_serror_rate',
 'service_http',
 'dst_host_same_srv_rate',
 'protocol_icmp',
 'dst_host_srv_serror_rate']

In [49]:
U2R_fet = list(set(RFE_dos_col[2]).union(set(RFE_dos_col[1]), set(U2R_uf_col_train)))
U2R_fet

['num_compromised',
 'src_bytes',
 'diff_srv_rate',
 'flag_SF',
 'service_telnet',
 'service_private',
 'dst_host_srv_diff_host_rate',
 'dst_host_same_src_port_rate',
 'service_ecr_i',
 'hot',
 'wrong_fragment',
 'service_ftp_data',
 'count',
 'serror_rate',
 'dst_host_count',
 'dst_bytes',
 'same_srv_rate',
 'root_shell',
 'dst_host_srv_count',
 'flag_RSTR',
 'flag_S0',
 'dst_host_serror_rate',
 'num_file_creations',
 'service_http',
 'srv_diff_host_rate',
 'dst_host_same_srv_rate',
 'urgent',
 'num_shells',
 'protocol_icmp',
 'dst_host_srv_serror_rate']

In [50]:
import pickle as pkl

In [53]:
#save train features
flt_dos_df = dos_df[dos_fet]
flt_probe_df = probe_df[probe_fet]
flt_R2L_df = r2l_df[R2L_fet]
flt_U2R_df = u2r_df[U2R_fet]
df_dic = {'dos': flt_dos_df, 'probe': flt_probe_df, 'R2L': flt_R2L_df, 'U2R': flt_U2R_df}

with open("train.pkl", 'wb') as f:
    pkl.dump(df_dic, f)

label_dic = {'dos': Y_DoS, 'probe': Y_Probe, 'R2L': Y_R2L, 'U2R': Y_U2R}
with open("train_Y.pkl", 'wb') as ofile:
    pkl.dump(label_dic, ofile)

In [54]:
#save test features
flt_dos_test_df = Dos_df_test[dos_fet]
flt_probe_test_df = Probe_df_test[probe_fet]
flt_R2L_test_df = R2L_df_test[R2L_fet]
flt_U2R_test_df = U2R_df_test[U2R_fet]
df_test_dic = {'dos': flt_dos_test_df, 'probe': flt_probe_test_df, 'R2L': flt_R2L_test_df, 'U2R': flt_U2R_test_df}

with open("test.pkl", 'wb') as f:
    pkl.dump(df_test_dic, f)
    
test_label_dic = {'dos': Y_DoS_test, 'probe': Y_Probe_test, 'R2L': Y_R2L_test, 'U2R': Y_U2R_test}
with open("test_Y.pkl", 'wb') as ofile:
    pkl.dump(test_label_dic, ofile)

In [57]:
#Save important features
important_features = {'dos': dos_fet, 'probe': probe_fet, 'r2l': R2L_fet, 'u2r': U2R_fet}
with open("imp_fet.pkl", 'wb') as ofile:
    pkl.dump(important_features, ofile)